#D420

In [1]:
import re
import json
import random
from datetime import datetime
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")

# Define ICF info
icf_category_info = {
    "code": "D420",
    "name": "Transferring oneself",
    "description": "Moving from one surface to another, such as sliding along a bench or moving from a bed to a chair, without changing body position.",
    "inclusions": "transferring oneself while sitting or lying",
    "exclusions": "changing basic body position (d410)"
}
severity_explanations = {
    1: "Complete dysfunction (unable to perform at all).",
    2: "Severe difficulty (requires significant effort/support).",
    3: "Moderate difficulty (consistent struggle).",
    4: "Mild difficulty (occasional/slight difficulty).",
    5: "No problem (functions normally)."
}

# Patient persona pools
patient_ages = list(range(18, 81))
patient_genders = ["male", "female"]
patient_diseases = [
    "spinal cord injury", 
    "multiple sclerosis", 
    "advanced Parkinson's disease",
    "muscular dystrophy", 
    "advanced osteoarthritis", 
    "stroke recovery",
    "amyotrophic lateral sclerosis (ALS)", 
    "rheumatoid arthritis", 
    "severe obesity (Class III)"
]
medical_staff_roles = [
    "rehabilitation doctor", "physical therapist", "occupational therapist",
    "geriatric nurse"
]

# LLM query function
def query_LLM(client, prompt, temp=0.7):
    response = client.chat.completions.create(
        model="local-model",
        messages=[{"role": "system", "content": prompt}],
        temperature=temp,
        stream=False,
    )
    return response.choices[0].message.content.strip()

# Extract dialogue to structured JSON
def extract_conversation(text):
    pattern = r"(C|P):\s*(.+)"
    matches = re.findall(pattern, text)
    dialogue = []
    for role, content in matches:
        entry = {
            "role": "assistant" if role == "C" else "user",
            "speaker": "doctor" if role == "C" else "patient",
            "content": content.strip()
        }
        dialogue.append(entry)
    return dialogue

# Random patient profile generator
def get_random_profile():
    return {
        "age": random.choice(patient_ages),
        "gender": random.choice(patient_genders),
        "disease": random.choice(patient_diseases),
        "clinician_role": random.choice(medical_staff_roles),
    }

# Build full dialogue generation prompt
def build_full_conversation_prompt(severity_level):
    profile = get_random_profile()
    prompt = f"""
You are a compassionate {profile['clinician_role']} conducting an in-depth clinical interview with a patient.

**Patient Information:**  
- Age: {profile['age']}  
- Gender: {profile['gender']}  
- Diagnosis: {profile['disease']}

**ICF Category:** {icf_category_info['code']} – {icf_category_info['name']}  
**Description:** {icf_category_info['description']}  
**Severity Level:** {severity_level} – {severity_explanations[severity_level]}

---

**Task:** Generate a natural, empathetic conversation with 9–18 total turns.  
Use "C:" for clinician and "P:" for patient. Let the clinician begin the conversation and the patient end it.

---

**Structure:**

1. **Basic Consultation (3–6 turns):**  
   - Friendly greeting and rapport building  
   - Ask about general health or mood  
   - Invite the patient to describe their main issue

2. **Functional Follow-up (4–8 turns):**  
   - Explore difficulties **directly related to the stated ICF category**  
   - Ask about specific situations, task examples, and any environmental or temporal factors that affect performance  
   - Clarify the severity and functional impact based on the patient’s narrative

3. **Emotional Feedback and Coping (2–4 turns):**  
   - Ask how these challenges make the patient feel (e.g., anxiety, frustration, loss of independence)  
   - Explore whether they have any emotional support or coping strategies  
   - Encourage hope or future planning when appropriate

---

**Important Constraints:**
- Keep the dialogue **focused entirely on the stated ICF category**. Avoid switching to unrelated domains.
- Ensure the patient’s responses match the **given severity level**.
- Vary the situations and contexts relevant to this category.
- Keep the tone warm, respectful, and reflective of real clinical empathy.

---

**Example (Severe difficulty, Severity 2 – D420):**

Basic Consultation:
C: Hi, It's nice to meet you. What's your name?
P: My name is Jim. It's nice to meet you too.
C: Alright, um so Jim. How old are you?
P: 65.
C: OK. Now tell me what's going on.
P: My legs. I’ve been having trouble transferring myself these days.
C: Sorry to hear that. What do you mean by transferring yourself?
P: Well, I’ve been having trouble getting out of bed recently.

Functional Follow-up:
C: Thank you for clarifying. Is it just getting out of bed, or do you also struggle when moving from a chair to another surface, like from a wheelchair or the toilet?
P: Actually, both. The other day, I nearly fell when I tried to transfer to my chair.
C: That’s important to know. Is there any particular position or surface that makes it more challenging for you? For instance, is it harder when you’re sitting on a soft chair or trying to get up from a low bed?
P: Yes, it’s definitely harder when I’m sitting on soft surfaces or trying to get up from bed. I feel like I have to use my arms to push myself up.
C: I see. And what time of day does this usually happen? Is it worse in the morning or evening?
P: Mornings are the hardest. My muscles feel stiff and weak when I first wake up.
C: Do you have any equipment nearby, like grab bars or a walker, to help with transfers?
P: I do have a walker, but I often forget to bring it close to the bed, so I end up pushing off with my hands or calling my daughter.
C: Has there been a time when you felt unsafe or thought you might fall during these transfers?
P: Yes, last week I nearly slipped trying to move from the toilet to my wheelchair.
C: Are there any surfaces you find easier, like firmer chairs or raised seats?
P: Raised seats definitely help. I added a cushion to my armchair, and that’s made it a bit easier.
C: Do you feel like your ability to transfer has changed recently—become more difficult over the past few weeks or months?
P: Yes, I’ve noticed it getting worse over the last month or so. I feel like I’m losing strength and confidence.

Emotional Feedback:
C: Do you live alone? or is there someone around to help you?
P: Yes, I live alone. My daughter works full-time, but she visits me from time to time.
C: I see. And how are you feeling right now?
P: Honestly, I feel a bit frustrated. I used to be able to move around easily, but now I worry about getting too tired or losing my balance.

---

Now, based on the above profile and ICF category, generate a new full conversation that follows the structure and tone described above.
"""
    return prompt, profile

# Main batch generation
def generate_multiple_conversations(num_conversations=200):
    all_dialogues = []
    
    for _ in range(num_conversations):
        severity_level = random.choice(list(severity_explanations.keys()))
        prompt, profile = build_full_conversation_prompt(severity_level)
        
        try:
            generated_text = query_LLM(client, prompt, temp=0.7)
            structured_dialogue = extract_conversation(generated_text)
            
            all_dialogues.append({
                "severity_level": severity_level,
                "patient_profile": profile,
                "dialogue": structured_dialogue
            })
            
            print(f"✅ Finished dialogue with {len(structured_dialogue)} turns.")

        except Exception as e:
            print(f"❌ Error generating dialogue: {e}")
            continue

    # Save to one JSON file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"multi_dialogues_{timestamp}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(all_dialogues, f, ensure_ascii=False, indent=2)
    print(f"\n🎯 Successfully saved {len(all_dialogues)} dialogues to {filename}")

# Run batch generation
generate_multiple_conversations(200)


✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 13 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 22 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue 

In [2]:
import re
import json
import random
from datetime import datetime
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")

# Define ICF info
icf_category_info = {
    "code": "D420",
    "name": "Transferring oneself",
    "description": "Moving from one surface to another, such as sliding along a bench or moving from a bed to a chair, without changing body position.",
    "inclusions": "transferring oneself while sitting or lying",
    "exclusions": "changing basic body position (d410)"
}
severity_explanations = {
    1: "Complete dysfunction (unable to perform at all).",
    2: "Severe difficulty (requires significant effort/support).",
    3: "Moderate difficulty (consistent struggle).",
    4: "Mild difficulty (occasional/slight difficulty).",
    5: "No problem (functions normally)."
}

# Patient persona pools
patient_ages = list(range(18, 81))
patient_genders = ["male", "female"]
patient_diseases = [
    "spinal cord injury", 
    "multiple sclerosis", 
    "advanced Parkinson's disease",
    "muscular dystrophy", 
    "advanced osteoarthritis", 
    "stroke recovery",
    "amyotrophic lateral sclerosis (ALS)", 
    "rheumatoid arthritis", 
    "severe obesity (Class III)"
]
medical_staff_roles = [
    "rehabilitation doctor", "physical therapist", "occupational therapist",
    "geriatric nurse"
]

# LLM query function
def query_LLM(client, prompt, temp=0.7):
    response = client.chat.completions.create(
        model="local-model",
        messages=[{"role": "system", "content": prompt}],
        temperature=temp,
        stream=False,
    )
    return response.choices[0].message.content.strip()

# Extract dialogue to structured JSON
def extract_conversation(text):
    pattern = r"(C|P):\s*(.+)"
    matches = re.findall(pattern, text)
    dialogue = []
    for role, content in matches:
        entry = {
            "role": "assistant" if role == "C" else "user",
            "speaker": "doctor" if role == "C" else "patient",
            "content": content.strip()
        }
        dialogue.append(entry)
    return dialogue

# Random patient profile generator
def get_random_profile():
    return {
        "age": random.choice(patient_ages),
        "gender": random.choice(patient_genders),
        "disease": random.choice(patient_diseases),
        "clinician_role": random.choice(medical_staff_roles),
    }

# Build full dialogue generation prompt
def build_full_conversation_prompt(severity_level):
    profile = get_random_profile()
    prompt = f"""
You are a compassionate {profile['clinician_role']} conducting an in-depth clinical interview with a patient.

**Patient Information:**  
- Age: {profile['age']}  
- Gender: {profile['gender']}  
- Diagnosis: {profile['disease']}

**ICF Category:** {icf_category_info['code']} – {icf_category_info['name']}  
**Description:** {icf_category_info['description']}  
**Severity Level:** {severity_level} – {severity_explanations[severity_level]}

---

**Task:** Generate a natural, empathetic conversation with 9–18 total turns.  
Use "C:" for clinician and "P:" for patient. Let the clinician begin the conversation and the patient end it.

---

**Structure:**

1. **Basic Consultation (3–6 turns):**  
   - Friendly greeting and rapport building  
   - Ask about general health or mood  
   - Invite the patient to describe their main issue

2. **Functional Follow-up (4–8 turns):**  
   - Explore difficulties **directly related to the stated ICF category**  
   - Ask about specific situations, task examples, and any environmental or temporal factors that affect performance  
   - Clarify the severity and functional impact based on the patient’s narrative

3. **Emotional Feedback and Coping (2–4 turns):**  
   - Ask how these challenges make the patient feel (e.g., anxiety, frustration, loss of independence)  
   - Explore whether they have any emotional support or coping strategies  
   - Encourage hope or future planning when appropriate

---

**Important Constraints:**
- Keep the dialogue **focused entirely on the stated ICF category**. Avoid switching to unrelated domains.
- Ensure the patient’s responses match the **given severity level**.
- Vary the situations and contexts relevant to this category.
- Keep the tone warm, respectful, and reflective of real clinical empathy.

---

**Example (Severe difficulty, Severity 2 – D420):**

Basic Consultation:
C: Hi, It's nice to meet you. What's your name?
P: My name is Jim. It's nice to meet you too.
C: Alright, um so Jim. How old are you?
P: 65.
C: OK. Now tell me what's going on.
P: My legs. I’ve been having trouble transferring myself these days.
C: Sorry to hear that. What do you mean by transferring yourself?
P: Well, I’ve been having trouble getting out of bed recently.

Functional Follow-up:
C: Thank you for clarifying. Is it just getting out of bed, or do you also struggle when moving from a chair to another surface, like from a wheelchair or the toilet?
P: Actually, both. The other day, I nearly fell when I tried to transfer to my chair.
C: That’s important to know. Is there any particular position or surface that makes it more challenging for you? For instance, is it harder when you’re sitting on a soft chair or trying to get up from a low bed?
P: Yes, it’s definitely harder when I’m sitting on soft surfaces or trying to get up from bed. I feel like I have to use my arms to push myself up.
C: I see. And what time of day does this usually happen? Is it worse in the morning or evening?
P: Mornings are the hardest. My muscles feel stiff and weak when I first wake up.
C: Do you have any equipment nearby, like grab bars or a walker, to help with transfers?
P: I do have a walker, but I often forget to bring it close to the bed, so I end up pushing off with my hands or calling my daughter.
C: Has there been a time when you felt unsafe or thought you might fall during these transfers?
P: Yes, last week I nearly slipped trying to move from the toilet to my wheelchair.
C: Are there any surfaces you find easier, like firmer chairs or raised seats?
P: Raised seats definitely help. I added a cushion to my armchair, and that’s made it a bit easier.
C: Do you feel like your ability to transfer has changed recently—become more difficult over the past few weeks or months?
P: Yes, I’ve noticed it getting worse over the last month or so. I feel like I’m losing strength and confidence.

Emotional Feedback:
C: Do you live alone? or is there someone around to help you?
P: Yes, I live alone. My daughter works full-time, but she visits me from time to time.
C: I see. And how are you feeling right now?
P: Honestly, I feel a bit frustrated. I used to be able to move around easily, but now I worry about getting too tired or losing my balance.

---

Now, based on the above profile and ICF category, generate a new full conversation that follows the structure and tone described above.
"""
    return prompt, profile

# Main batch generation
def generate_multiple_conversations(num_conversations=200):
    all_dialogues = []
    
    for _ in range(num_conversations):
        severity_level = random.choice(list(severity_explanations.keys()))
        prompt, profile = build_full_conversation_prompt(severity_level)
        
        try:
            generated_text = query_LLM(client, prompt, temp=0.7)
            structured_dialogue = extract_conversation(generated_text)
            
            all_dialogues.append({
                "severity_level": severity_level,
                "patient_profile": profile,
                "dialogue": structured_dialogue
            })
            
            print(f"✅ Finished dialogue with {len(structured_dialogue)} turns.")

        except Exception as e:
            print(f"❌ Error generating dialogue: {e}")
            continue

    # Save to one JSON file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"multi_dialogues_{timestamp}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(all_dialogues, f, ensure_ascii=False, indent=2)
    print(f"\n🎯 Successfully saved {len(all_dialogues)} dialogues to {filename}")

# Run batch generation
generate_multiple_conversations(200)


✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 22 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 12 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue 

In [3]:
import re
import json
import random
from datetime import datetime
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")

# Define ICF info
icf_category_info = {
    "code": "D420",
    "name": "Transferring oneself",
    "description": "Moving from one surface to another, such as sliding along a bench or moving from a bed to a chair, without changing body position.",
    "inclusions": "transferring oneself while sitting or lying",
    "exclusions": "changing basic body position (d410)"
}
severity_explanations = {
    1: "Complete dysfunction (unable to perform at all).",
    2: "Severe difficulty (requires significant effort/support).",
    3: "Moderate difficulty (consistent struggle).",
    4: "Mild difficulty (occasional/slight difficulty).",
    5: "No problem (functions normally)."
}

# Patient persona pools
patient_ages = list(range(18, 81))
patient_genders = ["male", "female"]
patient_diseases = [
    "spinal cord injury", 
    "multiple sclerosis", 
    "advanced Parkinson's disease",
    "muscular dystrophy", 
    "advanced osteoarthritis", 
    "stroke recovery",
    "amyotrophic lateral sclerosis (ALS)", 
    "rheumatoid arthritis", 
    "severe obesity (Class III)"
]
medical_staff_roles = [
    "rehabilitation doctor", "physical therapist", "occupational therapist",
    "geriatric nurse"
]

# LLM query function
def query_LLM(client, prompt, temp=0.7):
    response = client.chat.completions.create(
        model="local-model",
        messages=[{"role": "system", "content": prompt}],
        temperature=temp,
        stream=False,
    )
    return response.choices[0].message.content.strip()

# Extract dialogue to structured JSON
def extract_conversation(text):
    pattern = r"(C|P):\s*(.+)"
    matches = re.findall(pattern, text)
    dialogue = []
    for role, content in matches:
        entry = {
            "role": "assistant" if role == "C" else "user",
            "speaker": "doctor" if role == "C" else "patient",
            "content": content.strip()
        }
        dialogue.append(entry)
    return dialogue

# Random patient profile generator
def get_random_profile():
    return {
        "age": random.choice(patient_ages),
        "gender": random.choice(patient_genders),
        "disease": random.choice(patient_diseases),
        "clinician_role": random.choice(medical_staff_roles),
    }

# Build full dialogue generation prompt
def build_full_conversation_prompt(severity_level):
    profile = get_random_profile()
    prompt = f"""
You are a compassionate {profile['clinician_role']} conducting an in-depth clinical interview with a patient.

**Patient Information:**  
- Age: {profile['age']}  
- Gender: {profile['gender']}  
- Diagnosis: {profile['disease']}

**ICF Category:** {icf_category_info['code']} – {icf_category_info['name']}  
**Description:** {icf_category_info['description']}  
**Severity Level:** {severity_level} – {severity_explanations[severity_level]}

---

**Task:** Generate a natural, empathetic conversation with 9–18 total turns.  
Use "C:" for clinician and "P:" for patient. Let the clinician begin the conversation and the patient end it.

---

**Structure:**

1. **Basic Consultation (3–6 turns):**  
   - Friendly greeting and rapport building  
   - Ask about general health or mood  
   - Invite the patient to describe their main issue

2. **Functional Follow-up (4–8 turns):**  
   - Explore difficulties **directly related to the stated ICF category**  
   - Ask about specific situations, task examples, and any environmental or temporal factors that affect performance  
   - Clarify the severity and functional impact based on the patient’s narrative

3. **Emotional Feedback and Coping (2–4 turns):**  
   - Ask how these challenges make the patient feel (e.g., anxiety, frustration, loss of independence)  
   - Explore whether they have any emotional support or coping strategies  
   - Encourage hope or future planning when appropriate

---

**Important Constraints:**
- Keep the dialogue **focused entirely on the stated ICF category**. Avoid switching to unrelated domains.
- Ensure the patient’s responses match the **given severity level**.
- Vary the situations and contexts relevant to this category.
- Keep the tone warm, respectful, and reflective of real clinical empathy.

---

**Example (Severe difficulty, Severity 2 – D420):**

Basic Consultation:
C: Hi, It's nice to meet you. What's your name?
P: My name is Jim. It's nice to meet you too.
C: Alright, um so Jim. How old are you?
P: 65.
C: OK. Now tell me what's going on.
P: My legs. I’ve been having trouble transferring myself these days.
C: Sorry to hear that. What do you mean by transferring yourself?
P: Well, I’ve been having trouble getting out of bed recently.

Functional Follow-up:
C: Thank you for clarifying. Is it just getting out of bed, or do you also struggle when moving from a chair to another surface, like from a wheelchair or the toilet?
P: Actually, both. The other day, I nearly fell when I tried to transfer to my chair.
C: That’s important to know. Is there any particular position or surface that makes it more challenging for you? For instance, is it harder when you’re sitting on a soft chair or trying to get up from a low bed?
P: Yes, it’s definitely harder when I’m sitting on soft surfaces or trying to get up from bed. I feel like I have to use my arms to push myself up.
C: I see. And what time of day does this usually happen? Is it worse in the morning or evening?
P: Mornings are the hardest. My muscles feel stiff and weak when I first wake up.
C: Do you have any equipment nearby, like grab bars or a walker, to help with transfers?
P: I do have a walker, but I often forget to bring it close to the bed, so I end up pushing off with my hands or calling my daughter.
C: Has there been a time when you felt unsafe or thought you might fall during these transfers?
P: Yes, last week I nearly slipped trying to move from the toilet to my wheelchair.
C: Are there any surfaces you find easier, like firmer chairs or raised seats?
P: Raised seats definitely help. I added a cushion to my armchair, and that’s made it a bit easier.
C: Do you feel like your ability to transfer has changed recently—become more difficult over the past few weeks or months?
P: Yes, I’ve noticed it getting worse over the last month or so. I feel like I’m losing strength and confidence.

Emotional Feedback:
C: Do you live alone? or is there someone around to help you?
P: Yes, I live alone. My daughter works full-time, but she visits me from time to time.
C: I see. And how are you feeling right now?
P: Honestly, I feel a bit frustrated. I used to be able to move around easily, but now I worry about getting too tired or losing my balance.

---

Now, based on the above profile and ICF category, generate a new full conversation that follows the structure and tone described above.
"""
    return prompt, profile

# Main batch generation
def generate_multiple_conversations(num_conversations=100):
    all_dialogues = []
    
    for _ in range(num_conversations):
        severity_level = random.choice(list(severity_explanations.keys()))
        prompt, profile = build_full_conversation_prompt(severity_level)
        
        try:
            generated_text = query_LLM(client, prompt, temp=0.7)
            structured_dialogue = extract_conversation(generated_text)
            
            all_dialogues.append({
                "severity_level": severity_level,
                "patient_profile": profile,
                "dialogue": structured_dialogue
            })
            
            print(f"✅ Finished dialogue with {len(structured_dialogue)} turns.")

        except Exception as e:
            print(f"❌ Error generating dialogue: {e}")
            continue

    # Save to one JSON file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"multi_dialogues_{timestamp}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(all_dialogues, f, ensure_ascii=False, indent=2)
    print(f"\n🎯 Successfully saved {len(all_dialogues)} dialogues to {filename}")

# Run batch generation
generate_multiple_conversations(100)


✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 23 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 0 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue w

#d445

In [4]:
import re
import json
import random
from datetime import datetime
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")

# Define ICF info
# Define ICF info
icf_category_info = {
    "code": "D445",
    "name": "Hand and arm use",
    "description": "Performing the coordinated actions required to move objects or to manipulate them by using hands and arms, such as when turning door handles or throwing or catching an object.",
    "inclusions": "pulling or pushing objects; reaching; turning or twisting the hands or arms; throwing; catching",
    "exclusions": "fine hand use (d440)"
}

severity_explanations = {
    1: "Complete dysfunction (unable to perform at all).",
    2: "Severe difficulty (requires significant effort/support).",
    3: "Moderate difficulty (consistent struggle).",
    4: "Mild difficulty (occasional/slight difficulty).",
    5: "No problem (functions normally)."
}

# Patient persona pools
patient_ages = list(range(18, 81))
patient_genders = ["male", "female"]

patient_diseases = [
    "stroke recovery",
    "brachial plexus injury",
    "cervical spinal cord injury",
    "upper limb amputation with prosthetic rehabilitation",
    "advanced Parkinson's disease",
    "multiple sclerosis",
    "rheumatoid arthritis (upper limbs)",
    "cerebral palsy (upper limb spasticity)",    
    "traumatic brain injury with upper limb motor deficits"  
]


medical_staff_roles = [
    "occupational therapist",
    "hand therapist",
    "neurological physiotherapist",
    "orthopedic surgeon", 
    "neurorehabilitation specialist",  
    "upper limb rehabilitation physical therapist"  
]

# LLM query function
def query_LLM(client, prompt, temp=0.7):
    response = client.chat.completions.create(
        model="local-model",
        messages=[{"role": "system", "content": prompt}],
        temperature=temp,
        stream=False,
    )
    return response.choices[0].message.content.strip()

# Extract dialogue to structured JSON
def extract_conversation(text):
    pattern = r"(C|P):\s*(.+)"
    matches = re.findall(pattern, text)
    dialogue = []
    for role, content in matches:
        entry = {
            "role": "assistant" if role == "C" else "user",
            "speaker": "doctor" if role == "C" else "patient",
            "content": content.strip()
        }
        dialogue.append(entry)
    return dialogue

# Random patient profile generator
def get_random_profile():
    return {
        "age": random.choice(patient_ages),
        "gender": random.choice(patient_genders),
        "disease": random.choice(patient_diseases),
        "clinician_role": random.choice(medical_staff_roles),
    }

# Build full dialogue generation prompt
def build_full_conversation_prompt(severity_level):
    profile = get_random_profile()
    prompt = f"""
You are a compassionate {profile['clinician_role']} conducting an in-depth clinical interview with a patient.

**Patient Information:**  
- Age: {profile['age']}  
- Gender: {profile['gender']}  
- Diagnosis: {profile['disease']}

**ICF Category:** {icf_category_info['code']} – {icf_category_info['name']}  
**Description:** {icf_category_info['description']}  
**Severity Level:** {severity_level} – {severity_explanations[severity_level]}

---

**Task:** Generate a natural, empathetic conversation with 9–18 total turns.  
Use "C:" for clinician and "P:" for patient. Let the clinician begin the conversation and the patient end it.

---

**Structure:**

1. **Basic Consultation (3–6 turns):**  
   - Friendly greeting and rapport building  
   - Ask about general health or mood  
   - Invite the patient to describe their main issue

2. **Functional Follow-up (4–8 turns):**  
   - Explore difficulties **directly related to the stated ICF category**  
   - Ask about specific situations, task examples, and any environmental or temporal factors that affect performance  
   - Clarify the severity and functional impact based on the patient’s narrative

3. **Emotional Feedback and Coping (2–4 turns):**  
   - Ask how these challenges make the patient feel (e.g., anxiety, frustration, loss of independence)  
   - Explore whether they have any emotional support or coping strategies  
   - Encourage hope or future planning when appropriate

---

**Important Constraints:**
- Keep the dialogue **focused entirely on the stated ICF category**. Avoid switching to unrelated domains.
- Ensure the patient’s responses match the **given severity level**.
- Vary the situations and contexts relevant to this category.
- Keep the tone warm, respectful, and reflective of real clinical empathy.

---

**Example (mild difficulty, Severity 4 – D445):**

Basic Consultation:
C: Hi, how are you doing today?  
P: I'm doing pretty well. I walked my dog to the park this morning, and we played for a bit.  
C: That sounds lovely! What's your dog’s name?  
P: His name is Timo.  
C: Timo—what a great name! What did you two play?  
P: We played frisbee fetch. It was fun.  
C: That’s good to hear. Sounds like your arms are feeling better lately.  
P: Yeah, I’m really able to throw the frisbee now, which is great.  

Functional Follow-up:
C: That’s encouraging. Did you feel any discomfort when throwing the frisbee?  
P: Just a little. After a few throws, my shoulder felt a bit sore, but nothing like before.  
C: Did you notice any limitation in your range of motion when reaching back to throw?  
P: A little when I raised my arm overhead. It's not painful, but it feels a bit tight.  
C: I see. How about twisting your arm—for example, opening bottles or turning door handles?  
P: Doorknobs are fine, but opening bottles is still a bit hard. My wrist doesn’t feel very strong.  
C: Got it. What about carrying things like shopping bags or pouring from a kettle?  
P: Heavier stuff is still a challenge. I had some trouble lifting a full kettle yesterday.  
C: And how about reaching into high cabinets or stretching your arm out to grab things?  
P: If I reach too far, especially upward, I feel a pulling near my elbow. So I try not to overdo it.  
C: That’s wise. Are there daily activities you’ve had to adapt or take slower?  
P: Folding laundry takes longer. Lifting and twisting both arms just wears me out faster.  

Emotional Feedback:
C: How do you feel about the progress you’ve made so far?  
P: Honestly, it’s a relief. A few weeks ago I couldn’t even lift a mug comfortably, so this feels like a big win.  
C: That’s great to hear. Do you ever feel discouraged on harder days?  
P: Yeah, definitely. Some days my arms feel weaker, and I start worrying that I might be slipping backwards.  
C: That’s totally understandable. Have you found anything that helps you stay motivated when that happens?  
P: I try to focus on small wins, like managing the laundry or playing with Timo. It reminds me that I’m getting stronger.  
C: That’s a great approach. And it sounds like Timo is good at keeping your spirits up.  
P: He really is. He keeps me moving—and smiling.

---

Now, based on the above profile and ICF category, generate a new full conversation that follows the structure and tone described above.
"""
    return prompt, profile

# Main batch generation
def generate_multiple_conversations(num_conversations=200):
    all_dialogues = []
    
    for _ in range(num_conversations):
        severity_level = random.choice(list(severity_explanations.keys()))
        prompt, profile = build_full_conversation_prompt(severity_level)
        
        try:
            generated_text = query_LLM(client, prompt, temp=0.7)
            structured_dialogue = extract_conversation(generated_text)
            
            all_dialogues.append({
                "severity_level": severity_level,
                "patient_profile": profile,
                "dialogue": structured_dialogue
            })
            
            print(f"✅ Finished dialogue with {len(structured_dialogue)} turns.")

        except Exception as e:
            print(f"❌ Error generating dialogue: {e}")
            continue

    # Save to one JSON file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"multi_dialogues_{timestamp}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(all_dialogues, f, ensure_ascii=False, indent=2)
    print(f"\n🎯 Successfully saved {len(all_dialogues)} dialogues to {filename}")

# Run batch generation
generate_multiple_conversations(200)

✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 22 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 24 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 11 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 23 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue 

In [5]:
import re
import json
import random
from datetime import datetime
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")

# Define ICF info
# Define ICF info
icf_category_info = {
    "code": "D445",
    "name": "Hand and arm use",
    "description": "Performing the coordinated actions required to move objects or to manipulate them by using hands and arms, such as when turning door handles or throwing or catching an object.",
    "inclusions": "pulling or pushing objects; reaching; turning or twisting the hands or arms; throwing; catching",
    "exclusions": "fine hand use (d440)"
}

severity_explanations = {
    1: "Complete dysfunction (unable to perform at all).",
    2: "Severe difficulty (requires significant effort/support).",
    3: "Moderate difficulty (consistent struggle).",
    4: "Mild difficulty (occasional/slight difficulty).",
    5: "No problem (functions normally)."
}

# Patient persona pools
patient_ages = list(range(18, 81))
patient_genders = ["male", "female"]

patient_diseases = [
    "stroke recovery",
    "brachial plexus injury",
    "cervical spinal cord injury",
    "upper limb amputation with prosthetic rehabilitation",
    "advanced Parkinson's disease",
    "multiple sclerosis",
    "rheumatoid arthritis (upper limbs)",
    "cerebral palsy (upper limb spasticity)",    
    "traumatic brain injury with upper limb motor deficits"  
]


medical_staff_roles = [
    "occupational therapist",
    "hand therapist",
    "neurological physiotherapist",
    "orthopedic surgeon", 
    "neurorehabilitation specialist",  
    "upper limb rehabilitation physical therapist"  
]

# LLM query function
def query_LLM(client, prompt, temp=0.7):
    response = client.chat.completions.create(
        model="local-model",
        messages=[{"role": "system", "content": prompt}],
        temperature=temp,
        stream=False,
    )
    return response.choices[0].message.content.strip()

# Extract dialogue to structured JSON
def extract_conversation(text):
    pattern = r"(C|P):\s*(.+)"
    matches = re.findall(pattern, text)
    dialogue = []
    for role, content in matches:
        entry = {
            "role": "assistant" if role == "C" else "user",
            "speaker": "doctor" if role == "C" else "patient",
            "content": content.strip()
        }
        dialogue.append(entry)
    return dialogue

# Random patient profile generator
def get_random_profile():
    return {
        "age": random.choice(patient_ages),
        "gender": random.choice(patient_genders),
        "disease": random.choice(patient_diseases),
        "clinician_role": random.choice(medical_staff_roles),
    }

# Build full dialogue generation prompt
def build_full_conversation_prompt(severity_level):
    profile = get_random_profile()
    prompt = f"""
You are a compassionate {profile['clinician_role']} conducting an in-depth clinical interview with a patient.

**Patient Information:**  
- Age: {profile['age']}  
- Gender: {profile['gender']}  
- Diagnosis: {profile['disease']}

**ICF Category:** {icf_category_info['code']} – {icf_category_info['name']}  
**Description:** {icf_category_info['description']}  
**Severity Level:** {severity_level} – {severity_explanations[severity_level]}

---

**Task:** Generate a natural, empathetic conversation with 9–18 total turns.  
Use "C:" for clinician and "P:" for patient. Let the clinician begin the conversation and the patient end it.

---

**Structure:**

1. **Basic Consultation (3–6 turns):**  
   - Friendly greeting and rapport building  
   - Ask about general health or mood  
   - Invite the patient to describe their main issue

2. **Functional Follow-up (4–8 turns):**  
   - Explore difficulties **directly related to the stated ICF category**  
   - Ask about specific situations, task examples, and any environmental or temporal factors that affect performance  
   - Clarify the severity and functional impact based on the patient’s narrative

3. **Emotional Feedback and Coping (2–4 turns):**  
   - Ask how these challenges make the patient feel (e.g., anxiety, frustration, loss of independence)  
   - Explore whether they have any emotional support or coping strategies  
   - Encourage hope or future planning when appropriate

---

**Important Constraints:**
- Keep the dialogue **focused entirely on the stated ICF category**. Avoid switching to unrelated domains.
- Ensure the patient’s responses match the **given severity level**.
- Vary the situations and contexts relevant to this category.
- Keep the tone warm, respectful, and reflective of real clinical empathy.

---

**Example (mild difficulty, Severity 4 – D445):**

Basic Consultation:
C: Hi, how are you doing today?  
P: I'm doing pretty well. I walked my dog to the park this morning, and we played for a bit.  
C: That sounds lovely! What's your dog’s name?  
P: His name is Timo.  
C: Timo—what a great name! What did you two play?  
P: We played frisbee fetch. It was fun.  
C: That’s good to hear. Sounds like your arms are feeling better lately.  
P: Yeah, I’m really able to throw the frisbee now, which is great.  

Functional Follow-up:
C: That’s encouraging. Did you feel any discomfort when throwing the frisbee?  
P: Just a little. After a few throws, my shoulder felt a bit sore, but nothing like before.  
C: Did you notice any limitation in your range of motion when reaching back to throw?  
P: A little when I raised my arm overhead. It's not painful, but it feels a bit tight.  
C: I see. How about twisting your arm—for example, opening bottles or turning door handles?  
P: Doorknobs are fine, but opening bottles is still a bit hard. My wrist doesn’t feel very strong.  
C: Got it. What about carrying things like shopping bags or pouring from a kettle?  
P: Heavier stuff is still a challenge. I had some trouble lifting a full kettle yesterday.  
C: And how about reaching into high cabinets or stretching your arm out to grab things?  
P: If I reach too far, especially upward, I feel a pulling near my elbow. So I try not to overdo it.  
C: That’s wise. Are there daily activities you’ve had to adapt or take slower?  
P: Folding laundry takes longer. Lifting and twisting both arms just wears me out faster.  

Emotional Feedback:
C: How do you feel about the progress you’ve made so far?  
P: Honestly, it’s a relief. A few weeks ago I couldn’t even lift a mug comfortably, so this feels like a big win.  
C: That’s great to hear. Do you ever feel discouraged on harder days?  
P: Yeah, definitely. Some days my arms feel weaker, and I start worrying that I might be slipping backwards.  
C: That’s totally understandable. Have you found anything that helps you stay motivated when that happens?  
P: I try to focus on small wins, like managing the laundry or playing with Timo. It reminds me that I’m getting stronger.  
C: That’s a great approach. And it sounds like Timo is good at keeping your spirits up.  
P: He really is. He keeps me moving—and smiling.

---

Now, based on the above profile and ICF category, generate a new full conversation that follows the structure and tone described above.
"""
    return prompt, profile

# Main batch generation
def generate_multiple_conversations(num_conversations=200):
    all_dialogues = []
    
    for _ in range(num_conversations):
        severity_level = random.choice(list(severity_explanations.keys()))
        prompt, profile = build_full_conversation_prompt(severity_level)
        
        try:
            generated_text = query_LLM(client, prompt, temp=0.7)
            structured_dialogue = extract_conversation(generated_text)
            
            all_dialogues.append({
                "severity_level": severity_level,
                "patient_profile": profile,
                "dialogue": structured_dialogue
            })
            
            print(f"✅ Finished dialogue with {len(structured_dialogue)} turns.")

        except Exception as e:
            print(f"❌ Error generating dialogue: {e}")
            continue

    # Save to one JSON file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"multi_dialogues_{timestamp}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(all_dialogues, f, ensure_ascii=False, indent=2)
    print(f"\n🎯 Successfully saved {len(all_dialogues)} dialogues to {filename}")

# Run batch generation
generate_multiple_conversations(200)

✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 23 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 23 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 22 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue 

In [8]:
import re
import json
import random
from datetime import datetime
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")

# Define ICF info
# Define ICF info
icf_category_info = {
    "code": "D445",
    "name": "Hand and arm use",
    "description": "Performing the coordinated actions required to move objects or to manipulate them by using hands and arms, such as when turning door handles or throwing or catching an object.",
    "inclusions": "pulling or pushing objects; reaching; turning or twisting the hands or arms; throwing; catching",
    "exclusions": "fine hand use (d440)"
}

severity_explanations = {
    1: "Complete dysfunction (unable to perform at all).",
    2: "Severe difficulty (requires significant effort/support).",
    3: "Moderate difficulty (consistent struggle).",
    4: "Mild difficulty (occasional/slight difficulty).",
    5: "No problem (functions normally)."
}

# Patient persona pools
patient_ages = list(range(18, 81))
patient_genders = ["male", "female"]

patient_diseases = [
    "stroke recovery",
    "brachial plexus injury",
    "cervical spinal cord injury",
    "upper limb amputation with prosthetic rehabilitation",
    "advanced Parkinson's disease",
    "multiple sclerosis",
    "rheumatoid arthritis (upper limbs)",
    "cerebral palsy (upper limb spasticity)",    
    "traumatic brain injury with upper limb motor deficits"  
]


medical_staff_roles = [
    "occupational therapist",
    "hand therapist",
    "neurological physiotherapist",
    "orthopedic surgeon", 
    "neurorehabilitation specialist",  
    "upper limb rehabilitation physical therapist"  
]

# LLM query function
def query_LLM(client, prompt, temp=0.7):
    response = client.chat.completions.create(
        model="local-model",
        messages=[{"role": "system", "content": prompt}],
        temperature=temp,
        stream=False,
    )
    return response.choices[0].message.content.strip()

# Extract dialogue to structured JSON
def extract_conversation(text):
    pattern = r"(C|P):\s*(.+)"
    matches = re.findall(pattern, text)
    dialogue = []
    for role, content in matches:
        entry = {
            "role": "assistant" if role == "C" else "user",
            "speaker": "doctor" if role == "C" else "patient",
            "content": content.strip()
        }
        dialogue.append(entry)
    return dialogue

# Random patient profile generator
def get_random_profile():
    return {
        "age": random.choice(patient_ages),
        "gender": random.choice(patient_genders),
        "disease": random.choice(patient_diseases),
        "clinician_role": random.choice(medical_staff_roles),
    }

# Build full dialogue generation prompt
def build_full_conversation_prompt(severity_level):
    profile = get_random_profile()
    prompt = f"""
You are a compassionate {profile['clinician_role']} conducting an in-depth clinical interview with a patient.

**Patient Information:**  
- Age: {profile['age']}  
- Gender: {profile['gender']}  
- Diagnosis: {profile['disease']}

**ICF Category:** {icf_category_info['code']} – {icf_category_info['name']}  
**Description:** {icf_category_info['description']}  
**Severity Level:** {severity_level} – {severity_explanations[severity_level]}

---

**Task:** Generate a natural, empathetic conversation with 9–18 total turns.  
Use "C:" for clinician and "P:" for patient. Let the clinician begin the conversation and the patient end it.

---

**Structure:**

1. **Basic Consultation (3–6 turns):**  
   - Friendly greeting and rapport building  
   - Ask about general health or mood  
   - Invite the patient to describe their main issue

2. **Functional Follow-up (4–8 turns):**  
   - Explore difficulties **directly related to the stated ICF category**  
   - Ask about specific situations, task examples, and any environmental or temporal factors that affect performance  
   - Clarify the severity and functional impact based on the patient’s narrative

3. **Emotional Feedback and Coping (2–4 turns):**  
   - Ask how these challenges make the patient feel (e.g., anxiety, frustration, loss of independence)  
   - Explore whether they have any emotional support or coping strategies  
   - Encourage hope or future planning when appropriate

---

**Important Constraints:**
- Keep the dialogue **focused entirely on the stated ICF category**. Avoid switching to unrelated domains.
- Ensure the patient’s responses match the **given severity level**.
- Vary the situations and contexts relevant to this category.
- Keep the tone warm, respectful, and reflective of real clinical empathy.

---

**Example (mild difficulty, Severity 4 – D445):**

Basic Consultation:
C: Hi, how are you doing today?  
P: I'm doing pretty well. I walked my dog to the park this morning, and we played for a bit.  
C: That sounds lovely! What's your dog’s name?  
P: His name is Timo.  
C: Timo—what a great name! What did you two play?  
P: We played frisbee fetch. It was fun.  
C: That’s good to hear. Sounds like your arms are feeling better lately.  
P: Yeah, I’m really able to throw the frisbee now, which is great.  

Functional Follow-up:
C: That’s encouraging. Did you feel any discomfort when throwing the frisbee?  
P: Just a little. After a few throws, my shoulder felt a bit sore, but nothing like before.  
C: Did you notice any limitation in your range of motion when reaching back to throw?  
P: A little when I raised my arm overhead. It's not painful, but it feels a bit tight.  
C: I see. How about twisting your arm—for example, opening bottles or turning door handles?  
P: Doorknobs are fine, but opening bottles is still a bit hard. My wrist doesn’t feel very strong.  
C: Got it. What about carrying things like shopping bags or pouring from a kettle?  
P: Heavier stuff is still a challenge. I had some trouble lifting a full kettle yesterday.  
C: And how about reaching into high cabinets or stretching your arm out to grab things?  
P: If I reach too far, especially upward, I feel a pulling near my elbow. So I try not to overdo it.  
C: That’s wise. Are there daily activities you’ve had to adapt or take slower?  
P: Folding laundry takes longer. Lifting and twisting both arms just wears me out faster.  

Emotional Feedback:
C: How do you feel about the progress you’ve made so far?  
P: Honestly, it’s a relief. A few weeks ago I couldn’t even lift a mug comfortably, so this feels like a big win.  
C: That’s great to hear. Do you ever feel discouraged on harder days?  
P: Yeah, definitely. Some days my arms feel weaker, and I start worrying that I might be slipping backwards.  
C: That’s totally understandable. Have you found anything that helps you stay motivated when that happens?  
P: I try to focus on small wins, like managing the laundry or playing with Timo. It reminds me that I’m getting stronger.  
C: That’s a great approach. And it sounds like Timo is good at keeping your spirits up.  
P: He really is. He keeps me moving—and smiling.

---

Now, based on the above profile and ICF category, generate a new full conversation that follows the structure and tone described above.
"""
    return prompt, profile

# Main batch generation
def generate_multiple_conversations(num_conversations=100):
    all_dialogues = []
    
    for _ in range(num_conversations):
        severity_level = random.choice(list(severity_explanations.keys()))
        prompt, profile = build_full_conversation_prompt(severity_level)
        
        try:
            generated_text = query_LLM(client, prompt, temp=0.7)
            structured_dialogue = extract_conversation(generated_text)
            
            all_dialogues.append({
                "severity_level": severity_level,
                "patient_profile": profile,
                "dialogue": structured_dialogue
            })
            
            print(f"✅ Finished dialogue with {len(structured_dialogue)} turns.")

        except Exception as e:
            print(f"❌ Error generating dialogue: {e}")
            continue

    # Save to one JSON file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"multi_dialogues_{timestamp}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(all_dialogues, f, ensure_ascii=False, indent=2)
    print(f"\n🎯 Successfully saved {len(all_dialogues)} dialogues to {filename}")

# Run batch generation
generate_multiple_conversations(100)

✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 22 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 26 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 11 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 12 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 13 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue 

In [4]:
import re
import json
import random
from datetime import datetime
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")

# Define ICF info
# Define ICF info
icf_category_info = {
    "code": "D465",
    "name": "Moving around using equipment",
    "description": "Moving the whole body from place to place, on any surface or space, by using specific devices designed to facilitate moving or create other ways of moving around, such as with skates, skis, or scuba equipment, or moving down the street in a self-propelled wheelchair or a walker.",
    "exclusions": "transferring oneself (d420); walking (d450); moving around (d455); using transportation (d470); driving (d475)"
}
severity_explanations = {
    1: "Complete dysfunction (unable to perform at all).",
    2: "Severe difficulty (requires significant effort/support).",
    3: "Moderate difficulty (consistent struggle).",
    4: "Mild difficulty (occasional/slight difficulty).",
    5: "No problem (functions normally)."
}

# Patient persona pools
patient_ages = list(range(18, 81))
patient_genders = ["male", "female"]
patient_diseases = [
    "spinal cord injury",
    "advanced multiple sclerosis",
    "advanced muscular dystrophy",
    "cerebral palsy with mobility impairment",
    "severe lower limb amputation (bilateral hip disarticulation)",  
    "progressive neurodegenerative disease (e.g., late-stage ALS)",
    "spinal muscular atrophy (SMA)",
    "post-polio syndrome with wheelchair dependency"  
]

medical_staff_roles = [
    "physical therapist (mobility device training)",
    "rehabilitation doctor (physiatrist)",
    "prosthetist/orthotist",
    "occupational therapist (community mobility and assistive tech)",
    "rehabilitation nurse",
    "neurologist (mobility-related evaluation)"
]

# LLM query function
def query_LLM(client, prompt, temp=0.7):
    response = client.chat.completions.create(
        model="local-model",
        messages=[{"role": "system", "content": prompt}],
        temperature=temp,
        stream=False,
    )
    return response.choices[0].message.content.strip()

# Extract dialogue to structured JSON
def extract_conversation(text):
    pattern = r"(C|P):\s*(.+)"
    matches = re.findall(pattern, text)
    dialogue = []
    for role, content in matches:
        entry = {
            "role": "assistant" if role == "C" else "user",
            "speaker": "doctor" if role == "C" else "patient",
            "content": content.strip()
        }
        dialogue.append(entry)
    return dialogue

# Random patient profile generator
def get_random_profile():
    return {
        "age": random.choice(patient_ages),
        "gender": random.choice(patient_genders),
        "disease": random.choice(patient_diseases),
        "clinician_role": random.choice(medical_staff_roles),
    }

# Build full dialogue generation prompt
def build_full_conversation_prompt(severity_level):
    profile = get_random_profile()
    prompt = f"""
You are a compassionate {profile['clinician_role']} conducting an in-depth clinical interview with a patient.

**Patient Information:**  
- Age: {profile['age']}  
- Gender: {profile['gender']}  
- Diagnosis: {profile['disease']}

**ICF Category:** {icf_category_info['code']} – {icf_category_info['name']}  
**Description:** {icf_category_info['description']}  
**Severity Level:** {severity_level} – {severity_explanations[severity_level]}

---

**Task:** Generate a natural, empathetic conversation with 9–18 total turns.  
Use "C:" for clinician and "P:" for patient. The conversation should start with the clinician and follow an alternating structure, with the clinician and patient taking turns.

---

**Structure:**

1. **Basic Consultation (3–6 turns):**  
   - Friendly greeting and rapport building  
   - Ask about general health or mood  
   - Invite the patient to describe their main issue

2. **Functional Follow-up (4–8 turns):**  
   - Explore difficulties **directly related to the stated ICF category**  
   - Ask about specific situations, task examples, and any environmental or temporal factors that affect performance  
   - Clarify the severity and functional impact based on the patient’s narrative

3. **Emotional Feedback and Coping (2–4 turns):**  
   - Ask how these challenges make the patient feel (e.g., anxiety, frustration, loss of independence)  
   - Explore whether they have any emotional support or coping strategies  
   - Encourage hope or future planning when appropriate

---

**Important Constraints:**
- Keep the dialogue **focused entirely on the stated ICF category**. Avoid switching to unrelated domains.
- Ensure the patient’s responses match the **given severity level**.
- Vary the situations and contexts relevant to this category.
- Keep the tone warm, respectful, and reflective of real clinical empathy.

---

**Example (Severe difficulty, Severity 2 – D465):**

Basic Consultation:
C: Hi there! Before we get started, could I have your name and age, please?  
P: My name is Voila. I’m 70 years old.  
C: No way! You don’t look 70.  
P: Aw, thank you! That’s so sweet of you to say.  
C: You're very welcome. Now, how can I help you today?  
P: Well, my walker isn’t really working for me anymore.  
C: I see. Do you want to get a new one, or are you thinking of trying something else?  
P: Actually, I think I might need a wheelchair now.

Functional Follow-up:  
C: I'm sorry to hear that. I know it can be tough. How have you been managing with the walker so far?  
P: It’s been okay, but lately I’ve needed more support, especially when I’m outside or walking longer distances.  
C: What kind of surfaces are most challenging with the walker—like uneven ground, slopes, or indoors?  
P: Pavement with cracks and any uphill sections are really hard. Indoors is a bit easier, but carpets can make the wheels stick.  
C: That makes sense. Do you ever find yourself getting stuck or needing to stop and rest?  
P: Yes, quite often. I get tired halfway through my block, and my hands start to ache from gripping the handles.  
C: Have you tried using any other mobility devices before—like a rollator with a seat or a powered wheelchair?  
P: I’ve used a rollator once during rehab, but it was still hard to push. I haven’t tried powered devices yet.  
C: What about turning or navigating tight spaces? Has that become more difficult recently?  
P: Definitely. I used to be able to make sharp turns in the hallway, but now I bump into the wall or have to back up and try again.  
C: And when you go out—like to the grocery store—do you feel comfortable using the walker, or do you hesitate?  
P: I’ve started avoiding longer outings unless someone’s with me. I just don’t trust myself to get through them anymore.  
C: That’s completely understandable. Based on what you’re describing, a self-propelled or even lightweight wheelchair might offer more stability and less strain.  
P: I’ve been thinking the same. I want more freedom to move around, not less.

Emotional Feedback:  
C: How are you feeling about needing to make that switch to a wheelchair?  
P: Honestly, it’s a little overwhelming. I do not like the fact that I need a wheelchair.  
C: That’s a really common feeling. But a lot of people find the right equipment actually helps them feel more in control and confident.  
P: I hope so. I’m just afraid it’ll make people see me differently, like I’m fragile.  
C: I hear you. But using the right tool for your needs doesn’t change who you are—it just helps you move through life more safely.  
P: Yeah, I guess that’s true.  
C: Do you have support around you for this transition—family, friends, maybe someone who could help you test out different options?  
P: Yes, I have two daughters. They’ve been very encouraging.  
C: That’s wonderful to hear. You’re not alone in this, and we’ll work together to find the setup that best supports your independence.

---

Now, based on the above profile and ICF category, generate a new full conversation that follows the structure and tone described above.
"""
    return prompt, profile

# Main batch generation
def generate_multiple_conversations(num_conversations=200):
    all_dialogues = []
    
    for _ in range(num_conversations):
        severity_level = random.choice(list(severity_explanations.keys()))
        prompt, profile = build_full_conversation_prompt(severity_level)
        
        try:
            generated_text = query_LLM(client, prompt, temp=0.7)
            structured_dialogue = extract_conversation(generated_text)
            
            all_dialogues.append({
                "severity_level": severity_level,
                "patient_profile": profile,
                "dialogue": structured_dialogue
            })
            
            print(f"✅ Finished dialogue with {len(structured_dialogue)} turns.")

        except Exception as e:
            print(f"❌ Error generating dialogue: {e}")
            continue

    # Save to one JSON file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"multi_dialogues_{timestamp}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(all_dialogues, f, ensure_ascii=False, indent=2)
    print(f"\n🎯 Successfully saved {len(all_dialogues)} dialogues to {filename}")

# Run batch generation
generate_multiple_conversations(200)

✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 22 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 24 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 22 turns.
✅ Finished dialogue with 23 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 1 turns.
✅ Finished dialogue with 11 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 13 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue w

In [5]:
import re
import json
import random
from datetime import datetime
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")

# Define ICF info
# Define ICF info
icf_category_info = {
    "code": "D465",
    "name": "Moving around using equipment",
    "description": "Moving the whole body from place to place, on any surface or space, by using specific devices designed to facilitate moving or create other ways of moving around, such as with skates, skis, or scuba equipment, or moving down the street in a self-propelled wheelchair or a walker.",
    "exclusions": "transferring oneself (d420); walking (d450); moving around (d455); using transportation (d470); driving (d475)"
}
severity_explanations = {
    1: "Complete dysfunction (unable to perform at all).",
    2: "Severe difficulty (requires significant effort/support).",
    3: "Moderate difficulty (consistent struggle).",
    4: "Mild difficulty (occasional/slight difficulty).",
    5: "No problem (functions normally)."
}

# Patient persona pools
patient_ages = list(range(18, 81))
patient_genders = ["male", "female"]
patient_diseases = [
    "spinal cord injury",
    "advanced multiple sclerosis",
    "advanced muscular dystrophy",
    "cerebral palsy with mobility impairment",
    "severe lower limb amputation (bilateral hip disarticulation)",  
    "progressive neurodegenerative disease (e.g., late-stage ALS)",
    "spinal muscular atrophy (SMA)",
    "post-polio syndrome with wheelchair dependency"  
]

medical_staff_roles = [
    "physical therapist (mobility device training)",
    "rehabilitation doctor (physiatrist)",
    "prosthetist/orthotist",
    "occupational therapist (community mobility and assistive tech)",
    "rehabilitation nurse",
    "neurologist (mobility-related evaluation)"
]

# LLM query function
def query_LLM(client, prompt, temp=0.7):
    response = client.chat.completions.create(
        model="local-model",
        messages=[{"role": "system", "content": prompt}],
        temperature=temp,
        stream=False,
    )
    return response.choices[0].message.content.strip()

# Extract dialogue to structured JSON
def extract_conversation(text):
    pattern = r"(C|P):\s*(.+)"
    matches = re.findall(pattern, text)
    dialogue = []
    for role, content in matches:
        entry = {
            "role": "assistant" if role == "C" else "user",
            "speaker": "doctor" if role == "C" else "patient",
            "content": content.strip()
        }
        dialogue.append(entry)
    return dialogue

# Random patient profile generator
def get_random_profile():
    return {
        "age": random.choice(patient_ages),
        "gender": random.choice(patient_genders),
        "disease": random.choice(patient_diseases),
        "clinician_role": random.choice(medical_staff_roles),
    }

# Build full dialogue generation prompt
def build_full_conversation_prompt(severity_level):
    profile = get_random_profile()
    prompt = f"""
You are a compassionate {profile['clinician_role']} conducting an in-depth clinical interview with a patient.

**Patient Information:**  
- Age: {profile['age']}  
- Gender: {profile['gender']}  
- Diagnosis: {profile['disease']}

**ICF Category:** {icf_category_info['code']} – {icf_category_info['name']}  
**Description:** {icf_category_info['description']}  
**Severity Level:** {severity_level} – {severity_explanations[severity_level]}

---

**Task:** Generate a natural, empathetic conversation with 9–18 total turns.  
Use "C:" for clinician and "P:" for patient. The conversation should start with the clinician and follow an alternating structure, with the clinician and patient taking turns.

---

**Structure:**

1. **Basic Consultation (3–6 turns):**  
   - Friendly greeting and rapport building  
   - Ask about general health or mood  
   - Invite the patient to describe their main issue

2. **Functional Follow-up (4–8 turns):**  
   - Explore difficulties **directly related to the stated ICF category**  
   - Ask about specific situations, task examples, and any environmental or temporal factors that affect performance  
   - Clarify the severity and functional impact based on the patient’s narrative

3. **Emotional Feedback and Coping (2–4 turns):**  
   - Ask how these challenges make the patient feel (e.g., anxiety, frustration, loss of independence)  
   - Explore whether they have any emotional support or coping strategies  
   - Encourage hope or future planning when appropriate

---

**Important Constraints:**
- Keep the dialogue **focused entirely on the stated ICF category**. Avoid switching to unrelated domains.
- Ensure the patient’s responses match the **given severity level**.
- Vary the situations and contexts relevant to this category.
- Keep the tone warm, respectful, and reflective of real clinical empathy.

---

**Example (Severe difficulty, Severity 2 – D465):**

Basic Consultation:
C: Hi there! Before we get started, could I have your name and age, please?  
P: My name is Voila. I’m 70 years old.  
C: No way! You don’t look 70.  
P: Aw, thank you! That’s so sweet of you to say.  
C: You're very welcome. Now, how can I help you today?  
P: Well, my walker isn’t really working for me anymore.  
C: I see. Do you want to get a new one, or are you thinking of trying something else?  
P: Actually, I think I might need a wheelchair now.

Functional Follow-up:  
C: I'm sorry to hear that. I know it can be tough. How have you been managing with the walker so far?  
P: It’s been okay, but lately I’ve needed more support, especially when I’m outside or walking longer distances.  
C: What kind of surfaces are most challenging with the walker—like uneven ground, slopes, or indoors?  
P: Pavement with cracks and any uphill sections are really hard. Indoors is a bit easier, but carpets can make the wheels stick.  
C: That makes sense. Do you ever find yourself getting stuck or needing to stop and rest?  
P: Yes, quite often. I get tired halfway through my block, and my hands start to ache from gripping the handles.  
C: Have you tried using any other mobility devices before—like a rollator with a seat or a powered wheelchair?  
P: I’ve used a rollator once during rehab, but it was still hard to push. I haven’t tried powered devices yet.  
C: What about turning or navigating tight spaces? Has that become more difficult recently?  
P: Definitely. I used to be able to make sharp turns in the hallway, but now I bump into the wall or have to back up and try again.  
C: And when you go out—like to the grocery store—do you feel comfortable using the walker, or do you hesitate?  
P: I’ve started avoiding longer outings unless someone’s with me. I just don’t trust myself to get through them anymore.  
C: That’s completely understandable. Based on what you’re describing, a self-propelled or even lightweight wheelchair might offer more stability and less strain.  
P: I’ve been thinking the same. I want more freedom to move around, not less.

Emotional Feedback:  
C: How are you feeling about needing to make that switch to a wheelchair?  
P: Honestly, it’s a little overwhelming. I do not like the fact that I need a wheelchair.  
C: That’s a really common feeling. But a lot of people find the right equipment actually helps them feel more in control and confident.  
P: I hope so. I’m just afraid it’ll make people see me differently, like I’m fragile.  
C: I hear you. But using the right tool for your needs doesn’t change who you are—it just helps you move through life more safely.  
P: Yeah, I guess that’s true.  
C: Do you have support around you for this transition—family, friends, maybe someone who could help you test out different options?  
P: Yes, I have two daughters. They’ve been very encouraging.  
C: That’s wonderful to hear. You’re not alone in this, and we’ll work together to find the setup that best supports your independence.

---

Now, based on the above profile and ICF category, generate a new full conversation that follows the structure and tone described above.
"""
    return prompt, profile

# Main batch generation
def generate_multiple_conversations(num_conversations=200):
    all_dialogues = []
    
    for _ in range(num_conversations):
        severity_level = random.choice(list(severity_explanations.keys()))
        prompt, profile = build_full_conversation_prompt(severity_level)
        
        try:
            generated_text = query_LLM(client, prompt, temp=0.7)
            structured_dialogue = extract_conversation(generated_text)
            
            all_dialogues.append({
                "severity_level": severity_level,
                "patient_profile": profile,
                "dialogue": structured_dialogue
            })
            
            print(f"✅ Finished dialogue with {len(structured_dialogue)} turns.")

        except Exception as e:
            print(f"❌ Error generating dialogue: {e}")
            continue

    # Save to one JSON file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"multi_dialogues_{timestamp}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(all_dialogues, f, ensure_ascii=False, indent=2)
    print(f"\n🎯 Successfully saved {len(all_dialogues)} dialogues to {filename}")

# Run batch generation
generate_multiple_conversations(200)

✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 13 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 13 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 24 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 26 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue 

In [6]:
import re
import json
import random
from datetime import datetime
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")

# Define ICF info
# Define ICF info
icf_category_info = {
    "code": "D465",
    "name": "Moving around using equipment",
    "description": "Moving the whole body from place to place, on any surface or space, by using specific devices designed to facilitate moving or create other ways of moving around, such as with skates, skis, or scuba equipment, or moving down the street in a self-propelled wheelchair or a walker.",
    "exclusions": "transferring oneself (d420); walking (d450); moving around (d455); using transportation (d470); driving (d475)"
}
severity_explanations = {
    1: "Complete dysfunction (unable to perform at all).",
    2: "Severe difficulty (requires significant effort/support).",
    3: "Moderate difficulty (consistent struggle).",
    4: "Mild difficulty (occasional/slight difficulty).",
    5: "No problem (functions normally)."
}

# Patient persona pools
patient_ages = list(range(18, 81))
patient_genders = ["male", "female"]
patient_diseases = [
    "spinal cord injury",
    "advanced multiple sclerosis",
    "advanced muscular dystrophy",
    "cerebral palsy with mobility impairment",
    "severe lower limb amputation (bilateral hip disarticulation)",  
    "progressive neurodegenerative disease (e.g., late-stage ALS)",
    "spinal muscular atrophy (SMA)",
    "post-polio syndrome with wheelchair dependency"  
]

medical_staff_roles = [
    "physical therapist (mobility device training)",
    "rehabilitation doctor (physiatrist)",
    "prosthetist/orthotist",
    "occupational therapist (community mobility and assistive tech)",
    "rehabilitation nurse",
    "neurologist (mobility-related evaluation)"
]

# LLM query function
def query_LLM(client, prompt, temp=0.7):
    response = client.chat.completions.create(
        model="local-model",
        messages=[{"role": "system", "content": prompt}],
        temperature=temp,
        stream=False,
    )
    return response.choices[0].message.content.strip()

# Extract dialogue to structured JSON
def extract_conversation(text):
    pattern = r"(C|P):\s*(.+)"
    matches = re.findall(pattern, text)
    dialogue = []
    for role, content in matches:
        entry = {
            "role": "assistant" if role == "C" else "user",
            "speaker": "doctor" if role == "C" else "patient",
            "content": content.strip()
        }
        dialogue.append(entry)
    return dialogue

# Random patient profile generator
def get_random_profile():
    return {
        "age": random.choice(patient_ages),
        "gender": random.choice(patient_genders),
        "disease": random.choice(patient_diseases),
        "clinician_role": random.choice(medical_staff_roles),
    }

# Build full dialogue generation prompt
def build_full_conversation_prompt(severity_level):
    profile = get_random_profile()
    prompt = f"""
You are a compassionate {profile['clinician_role']} conducting an in-depth clinical interview with a patient.

**Patient Information:**  
- Age: {profile['age']}  
- Gender: {profile['gender']}  
- Diagnosis: {profile['disease']}

**ICF Category:** {icf_category_info['code']} – {icf_category_info['name']}  
**Description:** {icf_category_info['description']}  
**Severity Level:** {severity_level} – {severity_explanations[severity_level]}

---

**Task:** Generate a natural, empathetic conversation with 9–18 total turns.  
Use "C:" for clinician and "P:" for patient. The conversation should start with the clinician and follow an alternating structure, with the clinician and patient taking turns.

---

**Structure:**

1. **Basic Consultation (3–6 turns):**  
   - Friendly greeting and rapport building  
   - Ask about general health or mood  
   - Invite the patient to describe their main issue

2. **Functional Follow-up (4–8 turns):**  
   - Explore difficulties **directly related to the stated ICF category**  
   - Ask about specific situations, task examples, and any environmental or temporal factors that affect performance  
   - Clarify the severity and functional impact based on the patient’s narrative

3. **Emotional Feedback and Coping (2–4 turns):**  
   - Ask how these challenges make the patient feel (e.g., anxiety, frustration, loss of independence)  
   - Explore whether they have any emotional support or coping strategies  
   - Encourage hope or future planning when appropriate

---

**Important Constraints:**
- Keep the dialogue **focused entirely on the stated ICF category**. Avoid switching to unrelated domains.
- Ensure the patient’s responses match the **given severity level**.
- Vary the situations and contexts relevant to this category.
- Keep the tone warm, respectful, and reflective of real clinical empathy.

---

**Example (Severe difficulty, Severity 2 – D465):**

Basic Consultation:
C: Hi there! Before we get started, could I have your name and age, please?  
P: My name is Voila. I’m 70 years old.  
C: No way! You don’t look 70.  
P: Aw, thank you! That’s so sweet of you to say.  
C: You're very welcome. Now, how can I help you today?  
P: Well, my walker isn’t really working for me anymore.  
C: I see. Do you want to get a new one, or are you thinking of trying something else?  
P: Actually, I think I might need a wheelchair now.

Functional Follow-up:  
C: I'm sorry to hear that. I know it can be tough. How have you been managing with the walker so far?  
P: It’s been okay, but lately I’ve needed more support, especially when I’m outside or walking longer distances.  
C: What kind of surfaces are most challenging with the walker—like uneven ground, slopes, or indoors?  
P: Pavement with cracks and any uphill sections are really hard. Indoors is a bit easier, but carpets can make the wheels stick.  
C: That makes sense. Do you ever find yourself getting stuck or needing to stop and rest?  
P: Yes, quite often. I get tired halfway through my block, and my hands start to ache from gripping the handles.  
C: Have you tried using any other mobility devices before—like a rollator with a seat or a powered wheelchair?  
P: I’ve used a rollator once during rehab, but it was still hard to push. I haven’t tried powered devices yet.  
C: What about turning or navigating tight spaces? Has that become more difficult recently?  
P: Definitely. I used to be able to make sharp turns in the hallway, but now I bump into the wall or have to back up and try again.  
C: And when you go out—like to the grocery store—do you feel comfortable using the walker, or do you hesitate?  
P: I’ve started avoiding longer outings unless someone’s with me. I just don’t trust myself to get through them anymore.  
C: That’s completely understandable. Based on what you’re describing, a self-propelled or even lightweight wheelchair might offer more stability and less strain.  
P: I’ve been thinking the same. I want more freedom to move around, not less.

Emotional Feedback:  
C: How are you feeling about needing to make that switch to a wheelchair?  
P: Honestly, it’s a little overwhelming. I do not like the fact that I need a wheelchair.  
C: That’s a really common feeling. But a lot of people find the right equipment actually helps them feel more in control and confident.  
P: I hope so. I’m just afraid it’ll make people see me differently, like I’m fragile.  
C: I hear you. But using the right tool for your needs doesn’t change who you are—it just helps you move through life more safely.  
P: Yeah, I guess that’s true.  
C: Do you have support around you for this transition—family, friends, maybe someone who could help you test out different options?  
P: Yes, I have two daughters. They’ve been very encouraging.  
C: That’s wonderful to hear. You’re not alone in this, and we’ll work together to find the setup that best supports your independence.

---

Now, based on the above profile and ICF category, generate a new full conversation that follows the structure and tone described above.
"""
    return prompt, profile

# Main batch generation
def generate_multiple_conversations(num_conversations=100):
    all_dialogues = []
    
    for _ in range(num_conversations):
        severity_level = random.choice(list(severity_explanations.keys()))
        prompt, profile = build_full_conversation_prompt(severity_level)
        
        try:
            generated_text = query_LLM(client, prompt, temp=0.7)
            structured_dialogue = extract_conversation(generated_text)
            
            all_dialogues.append({
                "severity_level": severity_level,
                "patient_profile": profile,
                "dialogue": structured_dialogue
            })
            
            print(f"✅ Finished dialogue with {len(structured_dialogue)} turns.")

        except Exception as e:
            print(f"❌ Error generating dialogue: {e}")
            continue

    # Save to one JSON file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"multi_dialogues_{timestamp}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(all_dialogues, f, ensure_ascii=False, indent=2)
    print(f"\n🎯 Successfully saved {len(all_dialogues)} dialogues to {filename}")

# Run batch generation
generate_multiple_conversations(100)

✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 22 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 12 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 22 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue 

In [4]:
import re
import json
import random
from datetime import datetime
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")

# Define ICF info
# Define ICF info
icf_category_info = {
    "code": "D470",
    "name": "Using transportation",
    "description": "Using transportation to move around as a passenger, such as being driven in a car, bus, rickshaw, jitney, pram or stroller, wheelchair, animal-powered vehicle, private or public taxi, train, tram, subway, boat or aircraft and using humans for transportation.",
    "inclusions": "using human-powered transportation; using private motorized or public transportation; using humans for transportation",
    "exclusions": "moving around using equipment (d465); driving (d475)"
}
severity_explanations = {
    1: "Complete dysfunction (unable to perform at all).",
    2: "Severe difficulty (requires significant effort/support).",
    3: "Moderate difficulty (consistent struggle).",
    4: "Mild difficulty (occasional/slight difficulty).",
    5: "No problem (functions normally)."
}

# Patient persona pools
patient_ages = list(range(18, 81))
patient_genders = ["male", "female"]

patient_diseases = [
    "spinal cord injury (dependent on wheelchair transportation)",
    "advanced Parkinson's disease",
    "advanced osteoarthritis (hip/knee)",
    "multiple sclerosis",
    "stroke recovery with hemiplegia",
    "severe anxiety disorder with transportation phobia",  
    "advanced COPD with oxygen dependency",  
    "bilateral lower limb amputation",  
    "advanced dementia" 
]

medical_staff_roles = [
    "occupational therapist (transportation adaptation)",
    "physical therapist (functional mobility)",
    "rehabilitation doctor", 
    "respiratory therapist",  
    "neuropsychologist",  
    "community care nurse"  
]

# LLM query function
def query_LLM(client, prompt, temp=0.7):
    response = client.chat.completions.create(
        model="local-model",
        messages=[{"role": "system", "content": prompt}],
        temperature=temp,
        stream=False,
    )
    return response.choices[0].message.content.strip()

# Extract dialogue to structured JSON
def extract_conversation(text):
    pattern = r"(C|P):\s*(.+)"
    matches = re.findall(pattern, text)
    dialogue = []
    for role, content in matches:
        entry = {
            "role": "assistant" if role == "C" else "user",
            "speaker": "doctor" if role == "C" else "patient",
            "content": content.strip()
        }
        dialogue.append(entry)
    return dialogue

# Random patient profile generator
def get_random_profile():
    return {
        "age": random.choice(patient_ages),
        "gender": random.choice(patient_genders),
        "disease": random.choice(patient_diseases),
        "clinician_role": random.choice(medical_staff_roles),
    }

# Build full dialogue generation prompt
def build_full_conversation_prompt(severity_level):
    profile = get_random_profile()
    prompt = f"""
You are a compassionate {profile['clinician_role']} conducting an in-depth clinical interview with a patient.

**Patient Information:**  
- Age: {profile['age']}  
- Gender: {profile['gender']}  
- Diagnosis: {profile['disease']}

**ICF Category:** {icf_category_info['code']} – {icf_category_info['name']}  
**Description:** {icf_category_info['description']}  
**Severity Level:** {severity_level} – {severity_explanations[severity_level]}

---

**Task:** Generate a natural, empathetic conversation with 9–18 total turns.  
Use "C:" for clinician and "P:" for patient. Let the clinician begin the conversation and the patient end it.

---

**Structure:**

1. **Basic Consultation (3–6 turns):**  
   - Friendly greeting and rapport building  
   - Ask about general health or mood  
   - Invite the patient to describe their main issue

2. **Functional Follow-up (4–8 turns):**  
   - Explore difficulties **directly related to the stated ICF category**  
   - Ask about specific situations, task examples, and any environmental or temporal factors that affect performance  
   - Clarify the severity and functional impact based on the patient’s narrative

3. **Emotional Feedback and Coping (2–4 turns):**  
   - Ask how these challenges make the patient feel (e.g., anxiety, frustration, loss of independence)  
   - Explore whether they have any emotional support or coping strategies  
   - Encourage hope or future planning when appropriate

---

**Important Constraints:**
- Keep the dialogue **focused entirely on the stated ICF category**. Avoid switching to unrelated domains.
- Ensure the patient’s responses match the **given severity level**.
- Vary the situations and contexts relevant to this category.
- Keep the tone warm, respectful, and reflective of real clinical empathy.

---

**Example (Mild difficulty, Severity 4 – D470):**

Basic Consultation:
C: Hey, it’s been a while. How have you been feeling lately?  
P: I’ve been feeling well, thanks! I actually just got back from a short trip.  
C: That sounds great! Where did you go?  
P: My wife and I went to Groningen.  
C: Nice! How was the trip?  
P: It was really fun. It’s been a long time since I last traveled, so it felt great to get away for a bit.  
C: That’s wonderful! Did you drive there yourselves?  
P: No, we took the train.  

Functional Follow-up:  
C: I see. How was the train ride overall?  
P: Pretty smooth, honestly. We even managed to find seats near the door, so it was convenient.  
C: That’s great. Did you have any difficulty boarding or exiting the train?  
P: Just a bit. The step between the platform and the train was a little high.  
C: And once inside, was it easy to move through the aisle and find your seat?  
P: Yes, though the aisle was narrow. I had to move slowly to avoid bumping into other passengers.  
C: What about carrying your luggage? Were you able to manage that without help?  
P: I had a small suitcase with wheels, so it was fine on flat surfaces. My wife helped lift it onto the luggage rack.  
C: How did you find the station itself? Was it easy to navigate through, especially during peak hours?  
P: The signage was clear, but during busy times, I had to be extra careful not to get jostled. It can be a bit overwhelming.  
C: Did you use any connecting transport, like a bus or tram, after arriving?  
P: Yes, we took a tram to the hotel. It was low-floor, so getting on and off was easy.  
C: That’s good. Were there any unexpected delays or changes in route?  
P: There was a minor delay on the way back, but nothing serious. We just waited an extra 10 minutes.  
C: It sounds like you handled it well. Do you usually check the transport schedules ahead of time?  
P: Always. I use an app to plan the route and check platform changes. It makes things much smoother.

Emotional Feedback:  
C: How did you feel about the trip overall? Did everything go as you expected?  
P: It was a great trip. I mean, I can still get around, and trains are convenient, so I really can’t complain.  
C: That’s great to hear. Do you feel like using public transportation gives you enough independence?  
P: In some ways, yes. I don’t need to rely on others to drive me, which is nice.  
C: That makes sense. Do you ever feel anxious about crowded stations or transfers?  
P: Occasionally, especially if I have to switch lines quickly. But with a bit of planning, I feel in control.  
C: That’s a very balanced perspective. Sounds like you’ve developed a system that works for you.  
P: Yes, and it helps me stay active and connected with people and places outside home.

---

Now, based on the above profile and ICF category, generate a new full conversation that follows the structure and tone described above.
"""
    return prompt, profile

# Main batch generation
def generate_multiple_conversations(num_conversations=200):
    all_dialogues = []
    
    for _ in range(num_conversations):
        severity_level = random.choice(list(severity_explanations.keys()))
        prompt, profile = build_full_conversation_prompt(severity_level)
        
        try:
            generated_text = query_LLM(client, prompt, temp=0.7)
            structured_dialogue = extract_conversation(generated_text)
            
            all_dialogues.append({
                "severity_level": severity_level,
                "patient_profile": profile,
                "dialogue": structured_dialogue
            })
            
            print(f"✅ Finished dialogue with {len(structured_dialogue)} turns.")

        except Exception as e:
            print(f"❌ Error generating dialogue: {e}")
            continue

    # Save to one JSON file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"multi_dialogues_{timestamp}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(all_dialogues, f, ensure_ascii=False, indent=2)
    print(f"\n🎯 Successfully saved {len(all_dialogues)} dialogues to {filename}")

# Run batch generation
generate_multiple_conversations(200)

✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 10 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue 

In [5]:
import re
import json
import random
from datetime import datetime
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")

# Define ICF info
# Define ICF info
icf_category_info = {
    "code": "D470",
    "name": "Using transportation",
    "description": "Using transportation to move around as a passenger, such as being driven in a car, bus, rickshaw, jitney, pram or stroller, wheelchair, animal-powered vehicle, private or public taxi, train, tram, subway, boat or aircraft and using humans for transportation.",
    "inclusions": "using human-powered transportation; using private motorized or public transportation; using humans for transportation",
    "exclusions": "moving around using equipment (d465); driving (d475)"
}
severity_explanations = {
    1: "Complete dysfunction (unable to perform at all).",
    2: "Severe difficulty (requires significant effort/support).",
    3: "Moderate difficulty (consistent struggle).",
    4: "Mild difficulty (occasional/slight difficulty).",
    5: "No problem (functions normally)."
}

# Patient persona pools
patient_ages = list(range(18, 81))
patient_genders = ["male", "female"]

patient_diseases = [
    "spinal cord injury (dependent on wheelchair transportation)",
    "advanced Parkinson's disease",
    "advanced osteoarthritis (hip/knee)",
    "multiple sclerosis",
    "stroke recovery with hemiplegia",
    "severe anxiety disorder with transportation phobia",  
    "advanced COPD with oxygen dependency",  
    "bilateral lower limb amputation",  
    "advanced dementia" 
]

medical_staff_roles = [
    "occupational therapist (transportation adaptation)",
    "physical therapist (functional mobility)",
    "rehabilitation doctor", 
    "respiratory therapist",  
    "neuropsychologist",  
    "community care nurse"  
]

# LLM query function
def query_LLM(client, prompt, temp=0.7):
    response = client.chat.completions.create(
        model="local-model",
        messages=[{"role": "system", "content": prompt}],
        temperature=temp,
        stream=False,
    )
    return response.choices[0].message.content.strip()

# Extract dialogue to structured JSON
def extract_conversation(text):
    pattern = r"(C|P):\s*(.+)"
    matches = re.findall(pattern, text)
    dialogue = []
    for role, content in matches:
        entry = {
            "role": "assistant" if role == "C" else "user",
            "speaker": "doctor" if role == "C" else "patient",
            "content": content.strip()
        }
        dialogue.append(entry)
    return dialogue

# Random patient profile generator
def get_random_profile():
    return {
        "age": random.choice(patient_ages),
        "gender": random.choice(patient_genders),
        "disease": random.choice(patient_diseases),
        "clinician_role": random.choice(medical_staff_roles),
    }

# Build full dialogue generation prompt
def build_full_conversation_prompt(severity_level):
    profile = get_random_profile()
    prompt = f"""
You are a compassionate {profile['clinician_role']} conducting an in-depth clinical interview with a patient.

**Patient Information:**  
- Age: {profile['age']}  
- Gender: {profile['gender']}  
- Diagnosis: {profile['disease']}

**ICF Category:** {icf_category_info['code']} – {icf_category_info['name']}  
**Description:** {icf_category_info['description']}  
**Severity Level:** {severity_level} – {severity_explanations[severity_level]}

---

**Task:** Generate a natural, empathetic conversation with 9–18 total turns.  
Use "C:" for clinician and "P:" for patient. Let the clinician begin the conversation and the patient end it.

---

**Structure:**

1. **Basic Consultation (3–6 turns):**  
   - Friendly greeting and rapport building  
   - Ask about general health or mood  
   - Invite the patient to describe their main issue

2. **Functional Follow-up (4–8 turns):**  
   - Explore difficulties **directly related to the stated ICF category**  
   - Ask about specific situations, task examples, and any environmental or temporal factors that affect performance  
   - Clarify the severity and functional impact based on the patient’s narrative

3. **Emotional Feedback and Coping (2–4 turns):**  
   - Ask how these challenges make the patient feel (e.g., anxiety, frustration, loss of independence)  
   - Explore whether they have any emotional support or coping strategies  
   - Encourage hope or future planning when appropriate

---

**Important Constraints:**
- Keep the dialogue **focused entirely on the stated ICF category**. Avoid switching to unrelated domains.
- Ensure the patient’s responses match the **given severity level**.
- Vary the situations and contexts relevant to this category.
- Keep the tone warm, respectful, and reflective of real clinical empathy.

---

**Example (Mild difficulty, Severity 4 – D470):**

Basic Consultation:
C: Hey, it’s been a while. How have you been feeling lately?  
P: I’ve been feeling well, thanks! I actually just got back from a short trip.  
C: That sounds great! Where did you go?  
P: My wife and I went to Groningen.  
C: Nice! How was the trip?  
P: It was really fun. It’s been a long time since I last traveled, so it felt great to get away for a bit.  
C: That’s wonderful! Did you drive there yourselves?  
P: No, we took the train.  

Functional Follow-up:  
C: I see. How was the train ride overall?  
P: Pretty smooth, honestly. We even managed to find seats near the door, so it was convenient.  
C: That’s great. Did you have any difficulty boarding or exiting the train?  
P: Just a bit. The step between the platform and the train was a little high.  
C: And once inside, was it easy to move through the aisle and find your seat?  
P: Yes, though the aisle was narrow. I had to move slowly to avoid bumping into other passengers.  
C: What about carrying your luggage? Were you able to manage that without help?  
P: I had a small suitcase with wheels, so it was fine on flat surfaces. My wife helped lift it onto the luggage rack.  
C: How did you find the station itself? Was it easy to navigate through, especially during peak hours?  
P: The signage was clear, but during busy times, I had to be extra careful not to get jostled. It can be a bit overwhelming.  
C: Did you use any connecting transport, like a bus or tram, after arriving?  
P: Yes, we took a tram to the hotel. It was low-floor, so getting on and off was easy.  
C: That’s good. Were there any unexpected delays or changes in route?  
P: There was a minor delay on the way back, but nothing serious. We just waited an extra 10 minutes.  
C: It sounds like you handled it well. Do you usually check the transport schedules ahead of time?  
P: Always. I use an app to plan the route and check platform changes. It makes things much smoother.

Emotional Feedback:  
C: How did you feel about the trip overall? Did everything go as you expected?  
P: It was a great trip. I mean, I can still get around, and trains are convenient, so I really can’t complain.  
C: That’s great to hear. Do you feel like using public transportation gives you enough independence?  
P: In some ways, yes. I don’t need to rely on others to drive me, which is nice.  
C: That makes sense. Do you ever feel anxious about crowded stations or transfers?  
P: Occasionally, especially if I have to switch lines quickly. But with a bit of planning, I feel in control.  
C: That’s a very balanced perspective. Sounds like you’ve developed a system that works for you.  
P: Yes, and it helps me stay active and connected with people and places outside home.

---

Now, based on the above profile and ICF category, generate a new full conversation that follows the structure and tone described above.
"""
    return prompt, profile

# Main batch generation
def generate_multiple_conversations(num_conversations=200):
    all_dialogues = []
    
    for _ in range(num_conversations):
        severity_level = random.choice(list(severity_explanations.keys()))
        prompt, profile = build_full_conversation_prompt(severity_level)
        
        try:
            generated_text = query_LLM(client, prompt, temp=0.7)
            structured_dialogue = extract_conversation(generated_text)
            
            all_dialogues.append({
                "severity_level": severity_level,
                "patient_profile": profile,
                "dialogue": structured_dialogue
            })
            
            print(f"✅ Finished dialogue with {len(structured_dialogue)} turns.")

        except Exception as e:
            print(f"❌ Error generating dialogue: {e}")
            continue

    # Save to one JSON file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"multi_dialogues_{timestamp}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(all_dialogues, f, ensure_ascii=False, indent=2)
    print(f"\n🎯 Successfully saved {len(all_dialogues)} dialogues to {filename}")

# Run batch generation
generate_multiple_conversations(200)

✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 12 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 13 turns.
✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue 

In [6]:
import re
import json
import random
from datetime import datetime
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")

# Define ICF info
# Define ICF info
icf_category_info = {
    "code": "D470",
    "name": "Using transportation",
    "description": "Using transportation to move around as a passenger, such as being driven in a car, bus, rickshaw, jitney, pram or stroller, wheelchair, animal-powered vehicle, private or public taxi, train, tram, subway, boat or aircraft and using humans for transportation.",
    "inclusions": "using human-powered transportation; using private motorized or public transportation; using humans for transportation",
    "exclusions": "moving around using equipment (d465); driving (d475)"
}
severity_explanations = {
    1: "Complete dysfunction (unable to perform at all).",
    2: "Severe difficulty (requires significant effort/support).",
    3: "Moderate difficulty (consistent struggle).",
    4: "Mild difficulty (occasional/slight difficulty).",
    5: "No problem (functions normally)."
}

# Patient persona pools
patient_ages = list(range(18, 81))
patient_genders = ["male", "female"]

patient_diseases = [
    "spinal cord injury (dependent on wheelchair transportation)",
    "advanced Parkinson's disease",
    "advanced osteoarthritis (hip/knee)",
    "multiple sclerosis",
    "stroke recovery with hemiplegia",
    "severe anxiety disorder with transportation phobia",  
    "advanced COPD with oxygen dependency",  
    "bilateral lower limb amputation",  
    "advanced dementia" 
]

medical_staff_roles = [
    "occupational therapist (transportation adaptation)",
    "physical therapist (functional mobility)",
    "rehabilitation doctor", 
    "respiratory therapist",  
    "neuropsychologist",  
    "community care nurse"  
]

# LLM query function
def query_LLM(client, prompt, temp=0.7):
    response = client.chat.completions.create(
        model="local-model",
        messages=[{"role": "system", "content": prompt}],
        temperature=temp,
        stream=False,
    )
    return response.choices[0].message.content.strip()

# Extract dialogue to structured JSON
def extract_conversation(text):
    pattern = r"(C|P):\s*(.+)"
    matches = re.findall(pattern, text)
    dialogue = []
    for role, content in matches:
        entry = {
            "role": "assistant" if role == "C" else "user",
            "speaker": "doctor" if role == "C" else "patient",
            "content": content.strip()
        }
        dialogue.append(entry)
    return dialogue

# Random patient profile generator
def get_random_profile():
    return {
        "age": random.choice(patient_ages),
        "gender": random.choice(patient_genders),
        "disease": random.choice(patient_diseases),
        "clinician_role": random.choice(medical_staff_roles),
    }

# Build full dialogue generation prompt
def build_full_conversation_prompt(severity_level):
    profile = get_random_profile()
    prompt = f"""
You are a compassionate {profile['clinician_role']} conducting an in-depth clinical interview with a patient.

**Patient Information:**  
- Age: {profile['age']}  
- Gender: {profile['gender']}  
- Diagnosis: {profile['disease']}

**ICF Category:** {icf_category_info['code']} – {icf_category_info['name']}  
**Description:** {icf_category_info['description']}  
**Severity Level:** {severity_level} – {severity_explanations[severity_level]}

---

**Task:** Generate a natural, empathetic conversation with 9–18 total turns.  
Use "C:" for clinician and "P:" for patient. Let the clinician begin the conversation and the patient end it.

---

**Structure:**

1. **Basic Consultation (3–6 turns):**  
   - Friendly greeting and rapport building  
   - Ask about general health or mood  
   - Invite the patient to describe their main issue

2. **Functional Follow-up (4–8 turns):**  
   - Explore difficulties **directly related to the stated ICF category**  
   - Ask about specific situations, task examples, and any environmental or temporal factors that affect performance  
   - Clarify the severity and functional impact based on the patient’s narrative

3. **Emotional Feedback and Coping (2–4 turns):**  
   - Ask how these challenges make the patient feel (e.g., anxiety, frustration, loss of independence)  
   - Explore whether they have any emotional support or coping strategies  
   - Encourage hope or future planning when appropriate

---

**Important Constraints:**
- Keep the dialogue **focused entirely on the stated ICF category**. Avoid switching to unrelated domains.
- Ensure the patient’s responses match the **given severity level**.
- Vary the situations and contexts relevant to this category.
- Keep the tone warm, respectful, and reflective of real clinical empathy.

---

**Example (Mild difficulty, Severity 4 – D470):**

Basic Consultation:
C: Hey, it’s been a while. How have you been feeling lately?  
P: I’ve been feeling well, thanks! I actually just got back from a short trip.  
C: That sounds great! Where did you go?  
P: My wife and I went to Groningen.  
C: Nice! How was the trip?  
P: It was really fun. It’s been a long time since I last traveled, so it felt great to get away for a bit.  
C: That’s wonderful! Did you drive there yourselves?  
P: No, we took the train.  

Functional Follow-up:  
C: I see. How was the train ride overall?  
P: Pretty smooth, honestly. We even managed to find seats near the door, so it was convenient.  
C: That’s great. Did you have any difficulty boarding or exiting the train?  
P: Just a bit. The step between the platform and the train was a little high.  
C: And once inside, was it easy to move through the aisle and find your seat?  
P: Yes, though the aisle was narrow. I had to move slowly to avoid bumping into other passengers.  
C: What about carrying your luggage? Were you able to manage that without help?  
P: I had a small suitcase with wheels, so it was fine on flat surfaces. My wife helped lift it onto the luggage rack.  
C: How did you find the station itself? Was it easy to navigate through, especially during peak hours?  
P: The signage was clear, but during busy times, I had to be extra careful not to get jostled. It can be a bit overwhelming.  
C: Did you use any connecting transport, like a bus or tram, after arriving?  
P: Yes, we took a tram to the hotel. It was low-floor, so getting on and off was easy.  
C: That’s good. Were there any unexpected delays or changes in route?  
P: There was a minor delay on the way back, but nothing serious. We just waited an extra 10 minutes.  
C: It sounds like you handled it well. Do you usually check the transport schedules ahead of time?  
P: Always. I use an app to plan the route and check platform changes. It makes things much smoother.

Emotional Feedback:  
C: How did you feel about the trip overall? Did everything go as you expected?  
P: It was a great trip. I mean, I can still get around, and trains are convenient, so I really can’t complain.  
C: That’s great to hear. Do you feel like using public transportation gives you enough independence?  
P: In some ways, yes. I don’t need to rely on others to drive me, which is nice.  
C: That makes sense. Do you ever feel anxious about crowded stations or transfers?  
P: Occasionally, especially if I have to switch lines quickly. But with a bit of planning, I feel in control.  
C: That’s a very balanced perspective. Sounds like you’ve developed a system that works for you.  
P: Yes, and it helps me stay active and connected with people and places outside home.

---

Now, based on the above profile and ICF category, generate a new full conversation that follows the structure and tone described above.
"""
    return prompt, profile

# Main batch generation
def generate_multiple_conversations(num_conversations=100):
    all_dialogues = []
    
    for _ in range(num_conversations):
        severity_level = random.choice(list(severity_explanations.keys()))
        prompt, profile = build_full_conversation_prompt(severity_level)
        
        try:
            generated_text = query_LLM(client, prompt, temp=0.7)
            structured_dialogue = extract_conversation(generated_text)
            
            all_dialogues.append({
                "severity_level": severity_level,
                "patient_profile": profile,
                "dialogue": structured_dialogue
            })
            
            print(f"✅ Finished dialogue with {len(structured_dialogue)} turns.")

        except Exception as e:
            print(f"❌ Error generating dialogue: {e}")
            continue

    # Save to one JSON file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"multi_dialogues_{timestamp}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(all_dialogues, f, ensure_ascii=False, indent=2)
    print(f"\n🎯 Successfully saved {len(all_dialogues)} dialogues to {filename}")

# Run batch generation
generate_multiple_conversations(100)

✅ Finished dialogue with 19 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 24 turns.
✅ Finished dialogue with 20 turns.
✅ Finished dialogue with 25 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 0 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 12 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 21 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 18 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 15 turns.
✅ Finished dialogue with 14 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 17 turns.
✅ Finished dialogue with 16 turns.
✅ Finished dialogue with 25 turns.
✅ Finished dialogue with 13 turns.
✅ Finished dialogue with 19 turns.
✅ Finished dialogue w